In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

option_type = 'call'
ticker = 'SPY'
data = yf.download(ticker, start='2022-01-01', end='2025-03-15')['Close']['SPY']

daily_returns = data.pct_change().dropna()

## PARAMETROS

#Precio Spot
S0 = data.iloc[-1]
sigma = daily_returns.std() * np.sqrt(252)

# Tasa libre de riesgo
r = 0.10

# Días al vencimiento
T = 252
dt = 1 / T

K = 650

num_simulations = 100

# Choques aleatorios para rendimientos del subyacente
Z = np.random.normal(0, 1, (num_simulations, T ) )

# Matriz de precios simulados del subyacente
# N filas X Pasos + 1 (Por S0)
price_matrix = np.zeros( (num_simulations, T + 1 ) )

## PRICING

barrier_price = S0 * 1.2

for i in range (num_simulations):

    #Inicio de la trayectoria simulada
    price_path = [S0]
    barrier_breached = False

    for dia in range(1, T+1):
        St = price_path[-1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z[i, dia - 1])
        price_path.append(St)

        # Para este ejemplo, si ST es mayor a la barrera:
        # Es una opción up- and ....
        if St > barrier_price:
            barrier_breached = True
            break

    # Como si se alcanza la barrera, el precio es 0
    # Es una opción up-and-out
    if barrier_breached:
        price_matrix[i, :] = 0
    else:
        price_matrix[i, :] = price_path


if(option_type == 'put'):
    price_matrix[:-1] = np.where(price_matrix[:-1] == 0, K+1, price_matrix[:-1])


put_payoffs = np.maximum(  K - price_matrix[:-1], 0)
put_value = np.exp(-r * T) * np.mean(put_payoffs)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [2]:
put_value

3.5518606661249856e-09